In [4]:
%reload_ext autoreload

In [3]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [5]:
from quantum_tools.examples import symbolic_contexts
from quantum_tools.symmetries.workspace import get_contraction_elements
from quantum_tools.examples.prob_dists import *
from quantum_tools.utilities.job_queuer_async import JobContext
from quantum_tools.utilities.constants import *
from quantum_tools.config import *
from quantum_tools.visualization.hypergraph import *
from quantum_tools.visualization.transversal_inequalities import *
from quantum_tools.visualization.triangle_plot import *
from quantum_tools.hypergraph.hypergraph_transversals import *
from quantum_tools.hypergraph.inequalities import *
from quantum_tools.contexts.measurement import *
from quantum_tools.contexts.quantum_caller import *
# from quantum_tools.inflation.positive_linear_solve import *
from quantum_tools.contexts.state import *
from quantum_tools.contexts.quantum_context import *
from scipy import sparse, optimize
import numpy as np
from quantum_tools.rmt import rmt
import math
from scipy import io
from cmath import exp
import os
from itertools import product, combinations, permutations, combinations_with_replacement
from functools import reduce
from operator import mul, itemgetter

In [6]:
import matplotlib as mpl
mpl.rcParams['figure.max_open_warning'] = False

## Example
#### Taken From Wagon Wheel Antecedent 12 Canonical Minimalization

In [7]:
example = 'P_{ABC}(000)P_{C}(3) \\leq 2P_{ABC}(203)P_{C}(0) + 2P_{ABC}(303)P_{C}(0) + P_{ABC}(003)P_{C}(0) + P_{ABC}(013)P_{C}(0) + P_{ABC}(020)P_{C}(2) + P_{ABC}(022)P_{C}(2) + P_{ABC}(023)P_{C}(0) + P_{ABC}(023)P_{C}(2) + P_{ABC}(030)P_{C}(2) + P_{ABC}(031)P_{C}(2) + P_{ABC}(032)P_{C}(2) + P_{ABC}(033)P_{C}(0) + P_{ABC}(033)P_{C}(2) + P_{ABC}(103)P_{C}(0) + P_{ABC}(113)P_{C}(0) + P_{ABC}(123)P_{C}(0) + P_{ABC}(133)P_{C}(0) + P_{ABC}(200)P_{C}(0) + P_{ABC}(200)P_{C}(1) + P_{ABC}(200)P_{C}(2) + P_{ABC}(200)P_{C}(3) + P_{ABC}(201)P_{C}(0) + P_{ABC}(201)P_{C}(1) + P_{ABC}(201)P_{C}(2) + P_{ABC}(201)P_{C}(3) + P_{ABC}(203)P_{C}(1) + P_{ABC}(203)P_{C}(2) + P_{ABC}(203)P_{C}(3) + P_{ABC}(213)P_{C}(0) + P_{ABC}(223)P_{C}(0) + P_{ABC}(300)P_{C}(0) + P_{ABC}(300)P_{C}(1) + P_{ABC}(300)P_{C}(2) + P_{ABC}(300)P_{C}(3) + P_{ABC}(301)P_{C}(0) + P_{ABC}(301)P_{C}(1) + P_{ABC}(301)P_{C}(2) + P_{ABC}(301)P_{C}(3) + P_{ABC}(302)P_{C}(1) + P_{ABC}(303)P_{C}(1) + P_{ABC}(303)P_{C}(2) + P_{ABC}(303)P_{C}(3) + P_{ABC}(313)P_{C}(0)'

In [90]:
example

'P_{ABC}(000)P_{C}(3) \\leq 2P_{ABC}(203)P_{C}(0) + 2P_{ABC}(303)P_{C}(0) + P_{ABC}(003)P_{C}(0) + P_{ABC}(013)P_{C}(0) + P_{ABC}(020)P_{C}(2) + P_{ABC}(022)P_{C}(2) + P_{ABC}(023)P_{C}(0) + P_{ABC}(023)P_{C}(2) + P_{ABC}(030)P_{C}(2) + P_{ABC}(031)P_{C}(2) + P_{ABC}(032)P_{C}(2) + P_{ABC}(033)P_{C}(0) + P_{ABC}(033)P_{C}(2) + P_{ABC}(103)P_{C}(0) + P_{ABC}(113)P_{C}(0) + P_{ABC}(123)P_{C}(0) + P_{ABC}(133)P_{C}(0) + P_{ABC}(200)P_{C}(0) + P_{ABC}(200)P_{C}(1) + P_{ABC}(200)P_{C}(2) + P_{ABC}(200)P_{C}(3) + P_{ABC}(201)P_{C}(0) + P_{ABC}(201)P_{C}(1) + P_{ABC}(201)P_{C}(2) + P_{ABC}(201)P_{C}(3) + P_{ABC}(203)P_{C}(1) + P_{ABC}(203)P_{C}(2) + P_{ABC}(203)P_{C}(3) + P_{ABC}(213)P_{C}(0) + P_{ABC}(223)P_{C}(0) + P_{ABC}(300)P_{C}(0) + P_{ABC}(300)P_{C}(1) + P_{ABC}(300)P_{C}(2) + P_{ABC}(300)P_{C}(3) + P_{ABC}(301)P_{C}(0) + P_{ABC}(301)P_{C}(1) + P_{ABC}(301)P_{C}(2) + P_{ABC}(301)P_{C}(3) + P_{ABC}(302)P_{C}(1) + P_{ABC}(303)P_{C}(1) + P_{ABC}(303)P_{C}(2) + P_{ABC}(303)P_{C}(3) + P_{A

In [8]:
Latex(*[example])

In [20]:
example_lhs_modified = [
    'P_{ABC}(000)P_{C}(3)',
]
example_rhs_modified = [
    '2P_{ABC}(203)P_{C}(0)',
    '2P_{ABC}(303)P_{C}(0)',
    'P_{ABC}(003)P_{C}(0)',
    'P_{ABC}(013)P_{C}(0)',
    'P_{ABC}(020)P_{C}(2)',
    'P_{ABC}(022)P_{C}(2)',
    'P_{ABC}(023)P_{C}(0)',
    'P_{ABC}(023)P_{C}(2)',
    'P_{ABC}(030)P_{C}(2)',
    'P_{ABC}(031)P_{C}(2)',
    'P_{ABC}(032)P_{C}(2)',
    'P_{ABC}(033)P_{C}(0)',
    'P_{ABC}(033)P_{C}(2)',
    'P_{AC}(13)P_{C}(0)',
    'P_{ABC}(200)',
    'P_{ABC}(201)',
    'P_{ABC}(203)P_{C}(1)',
    'P_{ABC}(203)P_{C}(2)',
    'P_{ABC}(203)P_{C}(3)',
    'P_{ABC}(213)P_{C}(0)',
    'P_{ABC}(223)P_{C}(0)',
    'P_{ABC}(300)',
    'P_{ABC}(301)',
    'P_{ABC}(302)P_{C}(1)',
    'P_{ABC}(303)P_{C}(1)',
    'P_{ABC}(303)P_{C}(2)',
    'P_{ABC}(303)P_{C}(3)',
    'P_{ABC}(313)P_{C}(0)',
]

In [47]:
example_lhs_modified = [
    'P_{ABC}(000)P_{C}(3)',
    'P_{ABC}(021)P_{C}(2)',
    'P_{ABC}(202)',
    'P_{ABC}(302)',
    'P_{ABC}(233)P_{C}(0)',
    'P_{AC}(33)P_{C}(0)',
]
example_rhs_modified = [
    'P_{ABC}(303)P_{C}(0)',
    'P_{ABC}(313)P_{C}(0)',
    'P_{ABC}(302)P_{C}(1)',
    'P_{AB}(02)P_{C}(2)',
    'P_{AB}(03)P_{C}(2)',
    'P_{AB}(20)',
    'P_{AB}(30)',
    'P_{C}(3)P_{C}(0)',
]

In [91]:
' + '.join(example_lhs_modified) + ' \\leq ' + ' + '.join(example_rhs_modified)

'P_{ABC}(000)P_{C}(3) + P_{ABC}(021)P_{C}(2) + P_{ABC}(202) + P_{ABC}(302) + P_{ABC}(233)P_{C}(0) + P_{AC}(33)P_{C}(0) \\leq P_{ABC}(303)P_{C}(0) + P_{ABC}(313)P_{C}(0) + P_{ABC}(302)P_{C}(1) + P_{AB}(02)P_{C}(2) + P_{AB}(03)P_{C}(2) + P_{AB}(20) + P_{AB}(30) + P_{C}(3)P_{C}(0)'

#### Taken From Wagon Wheel Antecedent 12 NO Minimalization

In [50]:
example_full = 'P_{ABC}(000)P_{C}(3) \\leq 2P_{ABC}(001)P_{C}(0) + 2P_{ABC}(001)P_{C}(1) + 2P_{ABC}(001)P_{C}(2) + 2P_{ABC}(001)P_{C}(3) + 2P_{ABC}(002)P_{C}(0) + 2P_{ABC}(002)P_{C}(1) + 2P_{ABC}(002)P_{C}(2) + 2P_{ABC}(002)P_{C}(3) + 2P_{ABC}(003)P_{C}(1) + 2P_{ABC}(003)P_{C}(2) + 2P_{ABC}(003)P_{C}(3) + 2P_{ABC}(013)P_{C}(0) + 2P_{ABC}(023)P_{C}(0) + 2P_{ABC}(033)P_{C}(0) + 2P_{ABC}(103)P_{C}(0) + 2P_{ABC}(203)P_{C}(0) + 2P_{ABC}(303)P_{C}(0) + 3P_{ABC}(003)P_{C}(0) + P_{ABC}(011)P_{C}(0) + P_{ABC}(011)P_{C}(1) + P_{ABC}(011)P_{C}(2) + P_{ABC}(011)P_{C}(3) + P_{ABC}(012)P_{C}(0) + P_{ABC}(012)P_{C}(1) + P_{ABC}(012)P_{C}(2) + P_{ABC}(012)P_{C}(3) + P_{ABC}(013)P_{C}(1) + P_{ABC}(013)P_{C}(2) + P_{ABC}(013)P_{C}(3) + P_{ABC}(020)P_{C}(0) + P_{ABC}(020)P_{C}(1) + P_{ABC}(020)P_{C}(2) + P_{ABC}(020)P_{C}(3) + P_{ABC}(022)P_{C}(0) + P_{ABC}(022)P_{C}(1) + P_{ABC}(022)P_{C}(2) + P_{ABC}(022)P_{C}(3) + P_{ABC}(023)P_{C}(1) + P_{ABC}(023)P_{C}(2) + P_{ABC}(023)P_{C}(3) + P_{ABC}(030)P_{C}(0) + P_{ABC}(030)P_{C}(1) + P_{ABC}(030)P_{C}(2) + P_{ABC}(030)P_{C}(3) + P_{ABC}(031)P_{C}(2) + P_{ABC}(032)P_{C}(0) + P_{ABC}(032)P_{C}(1) + P_{ABC}(032)P_{C}(2) + P_{ABC}(032)P_{C}(3) + P_{ABC}(033)P_{C}(1) + P_{ABC}(033)P_{C}(2) + P_{ABC}(033)P_{C}(3) + P_{ABC}(101)P_{C}(0) + P_{ABC}(101)P_{C}(1) + P_{ABC}(101)P_{C}(2) + P_{ABC}(101)P_{C}(3) + P_{ABC}(102)P_{C}(0) + P_{ABC}(102)P_{C}(1) + P_{ABC}(102)P_{C}(2) + P_{ABC}(102)P_{C}(3) + P_{ABC}(103)P_{C}(1) + P_{ABC}(103)P_{C}(2) + P_{ABC}(103)P_{C}(3) + P_{ABC}(113)P_{C}(0) + P_{ABC}(123)P_{C}(0) + P_{ABC}(133)P_{C}(0) + P_{ABC}(200)P_{C}(0) + P_{ABC}(200)P_{C}(1) + P_{ABC}(200)P_{C}(2) + P_{ABC}(200)P_{C}(3) + P_{ABC}(201)P_{C}(0) + P_{ABC}(201)P_{C}(1) + P_{ABC}(201)P_{C}(2) + P_{ABC}(201)P_{C}(3) + P_{ABC}(203)P_{C}(1) + P_{ABC}(203)P_{C}(2) + P_{ABC}(203)P_{C}(3) + P_{ABC}(213)P_{C}(0) + P_{ABC}(223)P_{C}(0) + P_{ABC}(300)P_{C}(0) + P_{ABC}(300)P_{C}(1) + P_{ABC}(300)P_{C}(2) + P_{ABC}(300)P_{C}(3) + P_{ABC}(301)P_{C}(0) + P_{ABC}(301)P_{C}(1) + P_{ABC}(301)P_{C}(2) + P_{ABC}(301)P_{C}(3) + P_{ABC}(302)P_{C}(1) + P_{ABC}(303)P_{C}(1) + P_{ABC}(303)P_{C}(2) + P_{ABC}(303)P_{C}(3) + P_{ABC}(313)P_{C}(0)'

In [54]:
[a for a in example_full.split() if a != '+']

['P_{ABC}(000)P_{C}(3)',
 '\\leq',
 '2P_{ABC}(001)P_{C}(0)',
 '2P_{ABC}(001)P_{C}(1)',
 '2P_{ABC}(001)P_{C}(2)',
 '2P_{ABC}(001)P_{C}(3)',
 '2P_{ABC}(002)P_{C}(0)',
 '2P_{ABC}(002)P_{C}(1)',
 '2P_{ABC}(002)P_{C}(2)',
 '2P_{ABC}(002)P_{C}(3)',
 '2P_{ABC}(003)P_{C}(1)',
 '2P_{ABC}(003)P_{C}(2)',
 '2P_{ABC}(003)P_{C}(3)',
 '2P_{ABC}(013)P_{C}(0)',
 '2P_{ABC}(023)P_{C}(0)',
 '2P_{ABC}(033)P_{C}(0)',
 '2P_{ABC}(103)P_{C}(0)',
 '2P_{ABC}(203)P_{C}(0)',
 '2P_{ABC}(303)P_{C}(0)',
 '3P_{ABC}(003)P_{C}(0)',
 'P_{ABC}(011)P_{C}(0)',
 'P_{ABC}(011)P_{C}(1)',
 'P_{ABC}(011)P_{C}(2)',
 'P_{ABC}(011)P_{C}(3)',
 'P_{ABC}(012)P_{C}(0)',
 'P_{ABC}(012)P_{C}(1)',
 'P_{ABC}(012)P_{C}(2)',
 'P_{ABC}(012)P_{C}(3)',
 'P_{ABC}(013)P_{C}(1)',
 'P_{ABC}(013)P_{C}(2)',
 'P_{ABC}(013)P_{C}(3)',
 'P_{ABC}(020)P_{C}(0)',
 'P_{ABC}(020)P_{C}(1)',
 'P_{ABC}(020)P_{C}(2)',
 'P_{ABC}(020)P_{C}(3)',
 'P_{ABC}(022)P_{C}(0)',
 'P_{ABC}(022)P_{C}(1)',
 'P_{ABC}(022)P_{C}(2)',
 'P_{ABC}(022)P_{C}(3)',
 'P_{ABC}(023)P_{C}(1)

In [86]:
example_full_lhs_modified = [
    'P_{ABC}(000)P_{C}(3)',
    'P_{ABC}(003)P_{C}(0)',
    'P_{ABC}(103)P_{C}(0)',
    'P_{ABC}(000)',
    'P_{ABC}(202)',
    'P_{ABC}(302)',
    'P_{ABC}(100)',
]
example_full_rhs_modified = [
    'P_{B}(0)',
    'P_{AC}(03)P_{C}(0)',
    'P_{AC}(13)P_{C}(0)',
    'P_{AC}(02)',
    'P_{AC}(03)',
    'P_{BC}(03)P_{C}(0)',
    'P_{ABC}(011)',
    'P_{ABC}(020)',
    'P_{ABC}(030)',
    'P_{ABC}(001)',
    'P_{ABC}(031)P_{C}(2)',
    'P_{ABC}(223)P_{C}(0)',
    'P_{ABC}(302)P_{C}(1)',
    'P_{ABC}(213)P_{C}(0)',
    'P_{ABC}(313)P_{C}(0)'
]

In [92]:
' + '.join(example_full_lhs_modified) + ' \\leq ' + ' + '.join(example_full_rhs_modified)

'P_{ABC}(000)P_{C}(3) + P_{ABC}(003)P_{C}(0) + P_{ABC}(103)P_{C}(0) + P_{ABC}(000) + P_{ABC}(202) + P_{ABC}(302) + P_{ABC}(100) \\leq P_{B}(0) + P_{AC}(03)P_{C}(0) + P_{AC}(13)P_{C}(0) + P_{AC}(02) + P_{AC}(03) + P_{BC}(03)P_{C}(0) + P_{ABC}(011) + P_{ABC}(020) + P_{ABC}(030) + P_{ABC}(001) + P_{ABC}(031)P_{C}(2) + P_{ABC}(223)P_{C}(0) + P_{ABC}(302)P_{C}(1) + P_{ABC}(213)P_{C}(0) + P_{ABC}(313)P_{C}(0)'